In [1]:
# Import modules
import matplotlib.pyplot as plt
import numpy as np
# import tensorflow as tf
import pandas as pd
# from tensorflow import keras
# from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
# from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
# from keras.layers import CuDNNLSTM
# from keras.models import Sequential

In [2]:
# Initial figure settings
plt.rcParams['figure.figsize'] = [16,8]
plt.rcParams.update({'font.size': 18})
# Initial model training settings
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
# EDA class
class EDA:
  def __init__(self,data):
    self.data = data

  def data_sourcing(self):
    zeros = np.argwhere(self.data == 0)
    nans = np.argwhere(np.isnan(self.data))
    return [zeros, nans]

  def data_cleaning(self,interpolate=False):
    zeros, nans = self.data_sourcing()
    if (interpolate):
      pass
    else:
      remove = np.unique(np.concatenate([zeros[:,0], nans[:,0]]))
      return np.delete(self.data, remove, axis=0)

  def manipulation_analysis(self,frequency):
    diff_usd = [t - s for s, t in zip(self.data[::frequency], self.data[frequency::frequency])]
    diff_per = [(t - s)/s*100 for s, t in zip(self.data[::frequency], self.data[frequency::frequency])]
    sort_diff_usd = np.sort(diff_usd)
    sort_diff_per = np.sort(diff_per)
    abs_diff_usd = np.sort(np.absolute(diff_usd))
    abs_diff_per = np.sort(np.absolute(diff_per))
    upper = np.mean(abs_diff_per[-20:]) + np.std(abs_diff_per[-20:])
    lower = np.mean(abs_diff_per[-20:]) - np.std(abs_diff_per[-20:])
    return [lower, upper]

In [4]:
# Calculate Market Confidence Level
def calc_confidence_level(last,actual,mani_limits):
  lower = mani_limits[0]
  upper = mani_limits[1]
  diff = abs(last - actual)/last*100
  if (diff <= lower):
    return 100
  elif (diff >= upper):
    return 0
  else:
    return 100 - (diff-lower)/(upper-lower)*100

In [5]:
# Calculate Model Confidence Level
def calc_model_confidence(predicted,actual):
  diff = abs(predicted - actual)/actual*100
  return 100 - diff

In [6]:
# Print Decision function
def print_decision(dec):
  if (dec == 1):
    print ("  Decision is Buy")
  elif (dec == -1):
    print ("  Decision is Sell")
  else:
    print ("  Decision is Hold")
  return 0

In [7]:
# SVD Linear Regression Model
class SVD:
  def __init__(self):
    pass

  def split_validation_data(self,A,b,t,v):
    A_train = A[:-v-t]
    b_train = b[1:-v-t+1]
    A_test = A_train[:-t]
    b_test = b_train[:-t]
    A_vali = A[-v-1:-1]
    b_vali = b[-v-1:]
    return A_train, b_train, A_test, b_test, A_vali, b_vali
    
  def linear_regression(self,A_train,b_train):
    U, S, VT = np.linalg.svd(A_train,full_matrices=False)
    x = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ b_train
    return x

  def get_svd_error(self,A_test,b_test,x):
    error = mean_squared_error(A_test @ x, b_test)
    return error

  def normalize_data(self,A):
    A_mean = np.mean(A,axis=0)
    A_mean = A_mean.reshape(-1,1)
    A2 = A - np.ones((A.shape[0],1)) @ A_mean.T
    for j in range(A.shape[1]):
      A2std = np.std(A2[:,j])
      A2[:,j] = A2[:,j]/A2std
    return A2

In [8]:
# LSTM model
def lstm(data,seq_length,per_train):
  # Scale data
  scaler = MinMaxScaler()
  scaled_btc_price = scaler.fit_transform(data.reshape(-1,1))
  np.isnan(scaled_btc_price).any()

  # Create LSTM sequences
  def to_sequences(data, n):
    d = []
    for index in range(len(data) - n):
      d.append(data[index: index + n])
    return np.array(d)

  def preprocess(data_raw, n, per_train):
    data = to_sequences(data_raw, n)
    num_train = int(per_train * data.shape[0])
    A_train = data[:num_train, :-1, :]
    b_train = data[:num_train, -1, :]
    A_test = data[num_train:, :-1, :]
    b_test = data[num_train:, -1, :]
    return A_train, b_train, A_test, b_test

  A_train, b_train, A_test, b_test = preprocess(scaled_btc_price, seq_length, per_train)

  # Model
  DROPOUT = 0.2
  WINDOW_SIZE = seq_length - 1
  model = keras.Sequential()
  model.add(Bidirectional(CuDNNLSTM(WINDOW_SIZE, return_sequences=True),
    input_shape=(WINDOW_SIZE, A_train.shape[-1])))
  model.add(Dropout(rate=DROPOUT))
  model.add(Bidirectional(CuDNNLSTM((WINDOW_SIZE * 2), return_sequences=True)))
  model.add(Dropout(rate=DROPOUT))
  model.add(Bidirectional(CuDNNLSTM(WINDOW_SIZE, return_sequences=False)))
  model.add(Dense(units=1))
  model.add(Activation('linear'))

  # Train
  model.compile(
    loss='mean_squared_error',
    optimizer='adam'
  )
  BATCH_SIZE = 64
  history = model.fit(
    A_train,
    b_train,
    epochs=50,
    batch_size=BATCH_SIZE,
    shuffle=False,
    validation_split=0.1
  )

  # Model evaluation
  model.evaluate(A_test, b_test)

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

  # Prediction
  y_hat = model.predict(A_test)
  y_test_inverse = scaler.inverse_transform(b_test)
  y_hat_inverse = scaler.inverse_transform(y_hat)

  plt.plot(y_test_inverse, label="Actual Price", color='green')
  plt.plot(y_hat_inverse, label="Predicted Price", color='red')
  plt.title('Bitcoin price prediction')
  plt.xlabel('Time [days]')
  plt.ylabel('Price')
  plt.legend(loc='best')
  plt.show()

  return model

In [9]:
# Load data set
from google.colab import drive
drive.mount('/content/drive/')
!ls /content/drive

# Read data
dataframe = pd.ExcelFile('/content/drive/MyDrive/cryptoCurrencyPrediction/OnChainvsPriceproj712.xlsx')

Mounted at /content/drive/
MyDrive


In [10]:
# Curve fit for hourly
# Interpolation
def poly_fit(x,a,b,c,d): # generic cubic polynimal
  return a * x**3 + b * x**2 + c * x + d
def sin_fit(x,a,b,c,d,e): # generic quadratic polynimal + sin
  return a * np.sin(b - x) + c * x**2 + d * x + e

class CFH:
  def __init__(self,hourly):
    self.hourly = hourly

  def get_hourly_from_daily(self,daily,index):
    day = daily.loc[index]['Day']
    hours = self.hourly.loc[self.hourly['Day'] == day]
    return hours['BTC PRICE- USD'].to_numpy()

  def curve_fitting(self,data):
    x = np.arange(len(data)-1,47)
    y = np.arange(48)
    diff = np.setdiff1d(y,x)
    popt1, _ = curve_fit(poly_fit,diff,data)
    a,b,c,d = popt1
    predicted = poly_fit(diff,a,b,c,d)
    mse1 = mean_squared_error(predicted,data)
    popt2, _ = curve_fit(sin_fit,diff,data)
    a,b,c,d,e = popt2
    predicted = sin_fit(diff,a,b,c,d,e)
    mse2 = mean_squared_error(predicted,data)
    if (mse1 >= mse2):
      a,b,c,d,e = popt2
      return sin_fit(np.arange(24,48),a,b,c,d,e)
    else:
      a,b,c,d = popt1 
      return poly_fit(np.arange(24,48),a,b,c,d)

  def calc_local_min(self,data):
    return min(data), np.argmin(data)

  def calc_local_max(self,data):
    return max(data), np.argmax(data)

In [11]:
# Read daily data set
daily = pd.read_excel(dataframe, 'Daily Data', usecols="A:K")
daily = daily.rename(columns={'Unnamed: 0': 'Date'})
daily.sort_values('Date')
daily['Date'] = pd.to_datetime(daily['Date'])
daily['Day'] = daily['Date'].dt.date
# print (daily.head())

# Read hourly data set
hourly = pd.read_excel(dataframe, 'Hourly Data')
hourly.sort_values('timestamp')
hourly['timestamp'] = pd.to_datetime(hourly['timestamp'])
hourly['Day'] = hourly['timestamp'].dt.date
hourly['Hour'] = hourly['timestamp'].dt.time
# hourly.head()

In [12]:
# EDA for the daily data
arr_daily = daily.iloc[:,1:11].to_numpy()
eda = EDA(arr_daily)
arr_daily = eda.data_cleaning()
# Determine manipulation values
btc_daily = arr_daily[:,0]
eda = EDA(btc_daily)
daily_mani = eda.manipulation_analysis(1)
print ("Daily Manipulation Limits: ", daily_mani)

# EDA for the daily data
arr_hourly = hourly.iloc[:,1:7].to_numpy()
eda = EDA(arr_hourly)
arr_hourly = eda.data_cleaning()
# Determine manipulation values
btc_hourly = arr_hourly[:,0]
eda = EDA(btc_hourly)
hourly_mani = eda.manipulation_analysis(4)
print ("Hourly Manipulation Limits: ", hourly_mani)

Daily Manipulation Limits:  [8.27631528421832, 20.3127814578489]
Hourly Manipulation Limits:  [7.5966311419119545, 14.366813592086947]


In [17]:
# SVD Linear Regression for daily data
A = arr_daily[:,1:]
b = arr_daily[:,0] 
print (A.shape)
print (b.shape)

test = 15
validation = 60
nt = A.shape[0]-validation

(1139, 9)
(1139,)


In [18]:
# Split data into Train, Test and Dynamic Validation
svd = SVD()
A_train, b_train, A_test, b_test, A_dv, b_dv = svd.split_validation_data(A,b,test,validation)
# A_train_norm = svd.normalize_data(A_train)
daily_x = svd.linear_regression(A_train,b_train)
daily_error = svd.get_svd_error(A_test,b_test,daily_x)

A_tt = np.concatenate((A_train,A_test),axis=0)
b_tt = np.concatenate((b_train,b_test))
daily_x = svd.linear_regression(A_tt,b_tt)

# Calculate initial confidence level
predicted = A_tt[-1] @ daily_x # last predicted price from the test data
actual = b_tt[-1] # actual price for the same date
last = b_tt[-2] # actual price for the previous date
conf_model = calc_model_confidence(predicted,actual) # initial confidence level for the model
conf_market = calc_confidence_level(last,actual,daily_mani) # initial confidence level for the market
print ("Initial Market confidence level: ", conf_market)
print ("Initial Model confidence level: ", conf_model)

# # Run dynamic validation simulation
# # Trading decisions
# # buy,close -> profit # We only use this
# # short,close -> profit # This is also possible, but maybe not used commonly because risky
# # hold # we also use this

buys=[]
sells=[]
buy_indices=[]
sell_indices=[]
profits=[]
dec_counter=np.array([0,0,0]) # initialize Buy/Sell/Hold counter
last_dec = -1 # initialize decision as sell so that first decision is buy
for i in range(A_dv.shape[0]):
  print ("Validation Loop #",i+1)
  last = b_dv[i] # current actual price
  predicted = A_dv[i] @ daily_x # prediction for next price
  print ("  Current actual price: ", last)
  print ("  Predicted next price: ", predicted)
  print ("  Percentage difference: ", (predicted-last)/last*100)
  print ("  Actual next price: ", b_dv[i+1])
  print ("  Confidence Levels for Market: %d and Model: %d" % (conf_market, conf_model))
  # Add today's data to train
  print ("  Adding current data to the list and removing the oldest.")
  A_tt = np.delete(A_tt,(0),axis=0)
  b_tt = np.delete(b_tt,0)
  A_tt = np.append(A_tt,[A_dv[i]],axis=0)
  b_tt = np.append(b_tt,b_dv[i+1])
  # basic trading algorithm
  if (conf_market >= 50) and (conf_model >= 75):
    print ("  Market and Model are reliable.")
    if (predicted > last*1.0015) and (last_dec == -1):
      decision = 1 # buy if predicted is bigger than 1.015*last
      buy_indices.append(i)
      buys.append(last)
      dec_counter = dec_counter+np.array([1,0,0])
      last_dec = 1
    elif (predicted < last*0.9985) and (last_dec == 1):
      decision = -1 # sell if predicted is smaller than 0.985*last
      sell_indices.append(i)
      sells.append(last)
      dec_counter = dec_counter+np.array([0,1,0])
      last_dec = -1
      profits.append(sells[-1]-buys[-1])
    else:
      decision = 0 # hold
      dec_counter = dec_counter+np.array([0,0,1])
  else:
    print ("  Either Market or Model is not reliable.")
    #########################################
    # New Addition: If market or model is not reliable, close/sell the last buy action.
    if (last_dec == 1):
      decision = -1
      sells.append(last)
      sell_indices.append(i)
      dec_counter = dec_counter+np.array([0,1,0])
      last_dec = 0
      profits.append(sells[-1]-buys[-1])
    else:
      decision = 0
      dec_counter = dec_counter+np.array([0,0,1])
    #########################################
    # Re-train the model
    svd = SVD()
    daily_x = svd.linear_regression(A_tt,b_tt)
    print ("  Model is re-trained.")
  print_decision(decision)
  # Get actual tomorrow's price
  actual = b_dv[i+1] # tomorrow's actual price
  # Re-calculate the manipulation limits
  print ("  Calculating new manipulation limits")
  eda = EDA(b_tt)
  mani = eda.manipulation_analysis(1)
  # Re-calculate the confidence levels
  print ("  Re-calculating the confidence levels.")
  conf_market = calc_confidence_level(last,actual,mani)
  conf_model = calc_model_confidence(predicted,actual)
profits = np.array(profits)
# print ("Profit factor: ", sum(profits[profits > 0])/abs(sum(profits[profits < 0])))
print ("Buy Indices: ", buy_indices)
print ("Sell Indices: ", sell_indices)
print ("Total profit: ", sum(profits))
print ("Hold profit: ", b_dv[-1]-b_dv[0])
print ("Buy/Sell/Hold decison counts: ", dec_counter)


Initial Market confidence level:  100
Initial Model confidence level:  66.9494926572212
Validation Loop # 1
  Current actual price:  17827.3238934861
  Predicted next price:  23596.625384305145
  Percentage difference:  32.362128636295665
  Actual next price:  17371.0623816472
  Confidence Levels for Market: 100 and Model: 66
  Adding current data to the list and removing the oldest.
  Either Market or Model is not reliable.
  Model is re-trained.
  Decision is Hold
  Calculating new manipulation limits
  Re-calculating the confidence levels.
Validation Loop # 2
  Current actual price:  17371.0623816472
  Predicted next price:  23001.939698275113
  Percentage difference:  32.415273130197406
  Actual next price:  16626.2363396482
  Confidence Levels for Market: 100 and Model: 64
  Adding current data to the list and removing the oldest.
  Either Market or Model is not reliable.
  Model is re-trained.
  Decision is Hold
  Calculating new manipulation limits
  Re-calculating the confidenc

In [19]:
# Split data into Train, Test and Dynamic Validation
svd = SVD()
A_train, b_train, A_test, b_test, A_dv, b_dv = svd.split_validation_data(A,b,test,validation)
# A_train_norm = svd.normalize_data(A_train)
daily_x = svd.linear_regression(A_train,b_train)
daily_error = svd.get_svd_error(A_test,b_test,daily_x)

A_tt = np.concatenate((A_train,A_test),axis=0)
b_tt = np.concatenate((b_train,b_test))
daily_x = svd.linear_regression(A_tt,b_tt)

# Calculate initial confidence level
predicted = A_tt[-1] @ daily_x # last predicted price from the test data
actual = b_tt[-1] # actual price for the same date
last = b_tt[-2] # actual price for the previous date
conf_model = calc_model_confidence(predicted,actual) # initial confidence level for the model
conf_market = calc_confidence_level(last,actual,daily_mani) # initial confidence level for the market
print ("Initial Market confidence level: ", conf_market)
print ("Initial Model confidence level: ", conf_model)

# # Run dynamic validation simulation
# # Trading decisions
# # buy,close -> profit # We only use this
# # short,close -> profit # This is also possible, but maybe not used commonly because risky
# # hold # we also use this

buys=[]
sells=[]
buy_indices=[]
sell_indices=[]
profits=[]
dec_counter=np.array([0,0,0]) # initialize Buy/Sell/Hold counter
last_dec = -1 # initialize decision as sell so that first decision is buy
for i in range(A_dv.shape[0]):
  print ("Validation Loop #",i+1)
  last = b_dv[i] # current actual price
  predicted = A_dv[i] @ daily_x # prediction for next price
  print ("  Current actual price: ", last)
  print ("  Predicted next price: ", predicted)
  print ("  Percentage difference: ", (predicted-last)/last*100)
  print ("  Actual next price: ", b_dv[i+1])
  print ("  Confidence Levels for Market: %d and Model: %d" % (conf_market, conf_model))
  # Add today's data to train
  print ("  Adding current data to the list and removing the oldest.")
  A_tt = np.delete(A_tt,(0),axis=0)
  b_tt = np.delete(b_tt,0)
  A_tt = np.append(A_tt,[A_dv[i]],axis=0)
  b_tt = np.append(b_tt,b_dv[i+1])
  # basic trading algorithm
  if (conf_market >= 50) and (conf_model >= 75):
    print ("  Market and Model are reliable.")
    if (predicted > last*1.0015) and (last_dec == -1):
      decision = 1 # buy if predicted is bigger than 1.015*last

      # Curve fit hourly data to find local min
      cfh = CFH(hourly)
      hours = cfh.get_hourly_from_daily(daily,nt+i-1) # Get previous day hourly data
      next_hours = cfh.get_hourly_from_daily(daily,nt+i) # Get next day hourly data
      hours = np.append(hours,last) # add current price to previous day hourly data
      hours = np.append(hours,predicted) # add prediction to previous day hourly data
      for j in range(23):
        curve = cfh.curve_fitting(hours)
        min_value, index = cfh.calc_local_min(curve)
        if (index == j):
          buy_indices.append([i,j])
          buys.append(min_value)
          dec_counter = dec_counter+np.array([1,0,0])
          last_dec = 1
          break
        else:
          hours = np.insert(hours,-1,next_hours[j])

    elif (predicted < last*0.9985) and (last_dec == 1):
      decision = -1 # sell if predicted is smaller than 0.985*last

      # Curve fit hourly data to find local max
      cfh = CFH(hourly)
      hours = cfh.get_hourly_from_daily(daily,nt+i-1) # Get previous day hourly data
      next_hours = cfh.get_hourly_from_daily(daily,nt+i) # Get next day hourly data
      hours = np.append(hours,last) # add current price to previous day hourly data
      hours = np.append(hours,predicted) # add prediction to previous day hourly data
      for j in range(23):
        curve = cfh.curve_fitting(hours)
        max_value, index = cfh.calc_local_max(curve)
        if (index == j):
          sell_indices.append([i,j])
          sells.append(max_value)
          dec_counter = dec_counter+np.array([0,1,0])
          last_dec = -1
          profits.append(sells[-1]-buys[-1])
          break
        else:
          hours = np.insert(hours,-1,next_hours[j])
          
    else:
      decision = 0 # hold
      dec_counter = dec_counter+np.array([0,0,1])
  else:
    print ("  Either Market or Model is not reliable.")
    #########################################
    # New Addition: If market or model is not reliable, close/sell the last buy action.
    if (last_dec == 1):
      decision = -1
      sells.append(last)
      sell_indices.append(i)
      dec_counter = dec_counter+np.array([0,1,0])
      last_dec = 0
      profits.append(sells[-1]-buys[-1])
    else:
      decision = 0
      dec_counter = dec_counter+np.array([0,0,1])
    #########################################
    # Re-train the model
    svd = SVD()
    daily_x = svd.linear_regression(A_tt,b_tt)
    print ("  Model is re-trained.")
  print_decision(decision)
  # Get actual tomorrow's price
  actual = b_dv[i+1] # tomorrow's actual price
  # Re-calculate the manipulation limits
  print ("  Calculating new manipulation limits")
  eda = EDA(b_tt)
  mani = eda.manipulation_analysis(1)
  # Re-calculate the confidence levels
  print ("  Re-calculating the confidence levels.")
  conf_market = calc_confidence_level(last,actual,mani)
  conf_model = calc_model_confidence(predicted,actual)
profits = np.array(profits)
# print ("Profit factor: ", sum(profits[profits > 0])/abs(sum(profits[profits < 0])))
print ("Buy Indices: ", buy_indices)
print ("Sell Indices: ", sell_indices)
print ("Total profit: ", sum(profits))
print ("Hold profit: ", b_dv[-1]-b_dv[0])
print ("Buy/Sell/Hold decison counts: ", dec_counter)


Initial Market confidence level:  100
Initial Model confidence level:  66.9494926572212
Validation Loop # 1
  Current actual price:  17827.3238934861
  Predicted next price:  23596.625384305145
  Percentage difference:  32.362128636295665
  Actual next price:  17371.0623816472
  Confidence Levels for Market: 100 and Model: 66
  Adding current data to the list and removing the oldest.
  Either Market or Model is not reliable.
  Model is re-trained.
  Decision is Hold
  Calculating new manipulation limits
  Re-calculating the confidence levels.
Validation Loop # 2
  Current actual price:  17371.0623816472
  Predicted next price:  23001.939698275113
  Percentage difference:  32.415273130197406
  Actual next price:  16626.2363396482
  Confidence Levels for Market: 100 and Model: 64
  Adding current data to the list and removing the oldest.
  Either Market or Model is not reliable.
  Model is re-trained.
  Decision is Hold
  Calculating new manipulation limits
  Re-calculating the confidenc